In [1]:
import os
os.environ['USER_AGENT'] = 'MyApp/1.0 (Custom LangChain Application)'
import numpy as np
import pandas as pd

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader, CSVLoader, UnstructuredCSVLoader
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma

In [2]:
open_embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
hf_embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3')

In [3]:
def get_embedding(text, embeddings):
    return embeddings.embed_query(text)

def cos_sim(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

def return_answer_candidate(df, query, embeddings):
    query_embedding = get_embedding(query, embeddings)
    df['similarity'] = df.embedding.apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    top_3_doc = df.sort_values(by='similarity', ascending=False).head(3)

    return top_3_doc

In [4]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '부동산 시장이 점점 더 복잡해지고 있어요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 변동했어요',
]
df = pd.DataFrame(data, columns=['text'])

In [5]:
open_df = df.copy()
open_df['embedding'] = df.apply(lambda row: get_embedding(row.text, open_embeddings), axis=1)
open_df

,text,embedding
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ..."
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,..."
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009285671636462212, -0.014616581611335278,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.01740216836333275, -0.0030780902598053217,..."
4,저는 빠른 비트를 좋아해요,"[-0.037991348654031754, -0.013411780819296837,..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.017488501965999603, -0.02157001383602619, ..."


In [6]:
hf_df = df.copy()
hf_df['embedding'] = df.apply(lambda row: get_embedding(row.text, hf_embeddings), axis=1)
hf_df

,text,embedding
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,..."
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0..."
2,전통 시장에는 다양한 물품들을 팔아요,"[0.0001013833170873113, 0.0005807742127217352,..."
3,부동산 시장이 점점 더 복잡해지고 있어요,"[-0.0077268523164093494, 0.021975157782435417,..."
4,저는 빠른 비트를 좋아해요,"[-0.019412308931350708, 0.012223082594573498, ..."
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,..."


In [7]:
query = '과일 값이 비싸다.'
open_sim_result = return_answer_candidate(open_df, query, open_embeddings)
open_sim_result

,text,embedding,similarity
2,전통 시장에는 다양한 물품들을 팔아요,"[-0.009285671636462212, -0.014616581611335278,...",0.824507
1,시장 물가가 올랐어요,"[-0.000218817382119596, -0.035430148243904114,...",0.814506
0,주식 시장이 급등했어요,"[-0.013676190748810768, -0.03501933813095093, ...",0.805363


In [8]:
hf_sim_result = return_answer_candidate(hf_df, query, hf_embeddings)
hf_sim_result

,text,embedding,similarity
1,시장 물가가 올랐어요,"[0.003978083375841379, 0.02656499296426773, -0...",0.647129
5,최근 비트코인 가격이 많이 변동했어요,"[-0.026391346007585526, -0.004527552053332329,...",0.614578
0,주식 시장이 급등했어요,"[-0.02831065095961094, -0.0020167904440313578,...",0.578653


In [9]:
loader = WebBaseLoader(
    ['https://docs.langchain.com/oss/python/langchain/overview', 'https://docs.langchain.com/oss/python/langgraph/overview']
)
docs = loader.load()
print(docs[0].metadata, docs[1].metadata)

{'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'title': 'LangChain overview - Docs by LangChain', 'language': 'en'} {'source': 'https://docs.langchain.com/oss/python/langgraph/overview', 'title': 'LangGraph overview - Docs by LangChain', 'language': 'en'}


In [10]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
docs = loader.load()
print(len(docs))

84


In [11]:
print(docs[1].metadata)
print(docs[1].page_content[:100])

{'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': './data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 1, 'page_label': '2'}
1 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
 
 
Executive Summary 1 
 2024년 주택시장 하향 안정 전망, 금리와 공급, 그리고


In [12]:
loader = PDFPlumberLoader('./data/2024_KB_부동산_보고서_최종.pdf')
docs = loader.load()
print(len(docs))

84


In [13]:
print(docs[1].metadata)
print(docs[1].page_content[:100])

{'source': './data/2024_KB_부동산_보고서_최종.pdf', 'file_path': './data/2024_KB_부동산_보고서_최종.pdf', 'page': 1, 'total_pages': 84, 'Title': 'Morning Meeting', 'Author': '손은경', 'Creator': 'Microsoft® Word 2016', 'CreationDate': "D:20240304153001+09'00'", 'ModDate': "D:20240304153001+09'00'", 'Producer': 'Microsoft® Word 2016'}
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망
Ex ecutive Summary 1
 2024년 주택시장 하향 안정 전망, 금리와 공급, 그리고 정책 3대 변수 


In [14]:
loader = CSVLoader('./data/서울시_부동산_실거래가_정보.csv', encoding='utf-8')
docs = loader.load()
print(len(docs))

2001


In [15]:
print(docs[5].metadata)
print(docs[5].page_content)

{'source': './data/서울시_부동산_실거래가_정보.csv', 'row': 5}
﻿접수연도: 2024
자치구코드: 11410
자치구명: 서대문구
법정동코드: 11600
법정동명: 창천동
지번구분: 
지번구분명: 
본번: 
부번: 
건물명: 
계약일: 20241031
물건금액(만원): 340000
건물면적(㎡): 421.83
토지면적(㎡): 284
층: 
권리구분: 
취소일: 
건축년도: 2014
건물용도: 단독다가구
신고구분: 직거래
신고한 개업공인중개사 시군구명: 


In [16]:
loader = UnstructuredCSVLoader('./data/서울시_부동산_실거래가_정보.csv', encoding='utf-8', mode='elements')
docs = loader.load()
print(len(docs))

1


In [17]:
print(docs[0].metadata['text_as_html'][:100])

<table><tr><td>접수연도</td><td>자치구코드</td><td>자치구명</td><td>법정동코드</td><td>법정동명</td><td>지번구분</td><td>지번구분명


In [18]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([full_text])
for doc in docs:
    doc.metadata['source'] = '2024_KB_부동산_보고서_최종.pdf'
print(len(docs))

217


In [23]:
print(docs[1].page_content[-100:])
print('--'*20)
print(docs[2].page_content[:100])

공급 기반인 인허가물량까지 급감. 청약 수요 위축으로 분양 지연이 장기
화될 가능성이 높은 가운데 정부의 공급 정책 구체화가 매우 중요 
3 노후계획도시 특별법과 재건축 시장 영향
----------------------------------------
3 노후계획도시 특별법과 재건축 시장 영향 
 
2023년 말 국회를 통과한 「노후계획도시 특별법」 시행을 앞두고 당초 51곳이었던 대상 지역이 108곳으
로 확대. 단기적 효과는


In [26]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)
text_splitter = SemanticChunker(
    embeddings=hf_embeddings,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=70
)
docs = text_splitter.create_documents([full_text])
for doc in docs:
    doc.metadata['source'] = '2024_KB_부동산_보고서_최종.pdf'
print(len(docs))

286


In [29]:
print(docs[1].page_content)
print('--'*20)
print(docs[2].page_content[:300])

매수 수요 위축으로 주택 매매 거래량이 급감하면서 향후 주택 경기에 대한 부정적 
시각이 팽배하다. 무엇보다 여전히 높은 금리가 부담으로 작용하고 있다. 주택 경기 불황기에 고금리 
부담은 주택 수요를 크게 위축시킬 수밖에 없기 때문이다. 다만 주택시장의 주요 변수들의 상황에 따라 
소폭 반등 혹은 하락폭 확대 등의 방향이 정해질 것으로 보인다. 2024년 주택시장의 주요 변수는 공급과 금리다. 급격하게 위축된 주택 공급이 단기간에 증가하기는 
쉽지 않으나 정부의 공급 시그널이 지속된다면 일정 부분 해소가 될 가능성이 있다. 무엇보다 금리가 주요 
변수가 될 것이다. 기준금리 인하 시기와 인하 폭에 따라 주택 수요는 영향을 받을 수밖에 없기 때문이다.
----------------------------------------
한편, 수요 위축으로 거래가 급감한 상황에서 실수요자 금융 지원, 관련 규제 완화 등 수요 회복을 위한 
정부 정책도 중요한 변수가 될 전망이다.  7대 이슈를 통해 바라보는 2024년 주택시장 
1 역대 최저 수준이 지속되고 있는 주택 거래 
 
주택 매매 거래는 2022년에 이어 2년 연속 침체. 총선 이후 정책 불확실성 해소와 금리 인하로 인한 회
복 가능성이 있으나 일부 지역 수요 쏠림 현상과 금리 인하 속도가 더딜 경우 회복세는 제한적일 전망 
2 주택공급 급격한 감소로 인한 공급 부족 가능성 
 
분양물량과 함께 장기적
